# SISTEMAS DISTRIBUIDOS DE PROCESAMIENTOS DE DATOS 2

## PRÁCTICA FINAL T2

### AUTOR: Ester Cortés García

En esta primera parte de la práctica, se pide resolver los varios ejercicios utilizando un notebook 
de Jupyter con Spark SQL 

## Iniciar Spark

In [13]:
#import findspark
#findspark.init("/home/ester/spark/spark-2.2.1-bin-hadoop2.7")

import pyspark
from pyspark.sql import SparkSession
spark = (SparkSession.builder
    .master("local[*]")
    .config("spark.driver.cores", 1)
    .appName("understanding_sparksession")
    .getOrCreate() )
sc = spark.sparkContext
print(spark)
print(sc)

<SparkContext master=local[*] appName=understanding_sparksession>


## Definición del esquema

In [14]:
from pyspark.sql.types import *
#from pyspark.sql import functions
from pyspark.sql.functions import *
customSchema = StructType([StructField("Id", LongType(), True),
                           StructField("Parent_sys_id", StringType(), True),
                           StructField("Source", StringType(), True),
                           StructField("Mentions", StringType(), True),
                           StructField("Target", StringType(), True),
                           StructField("Name_source", StringType(), True),
                           StructField("Body", StringType(), True),
                           StructField("Pub_date", TimestampType(), True),
                           StructField("URLs", StringType(), True),
                           StructField("Tipe_action", StringType(), True),
                           StructField("Link", StringType(), True),
                           StructField("Has_link", ByteType(), True),
                           StructField("Has_picture", ByteType(), True),
                           StructField("Website", StringType(), True),
                           StructField("Country", StringType(), True),
                           StructField("Activity", LongType(), True),
                           StructField("Followers", LongType(), True),
                           StructField("Following", LongType(), True),
                           StructField("Location", StringType(), True)
                          ])

## Carga de datos 

In [15]:
events = spark.read.csv("/home/ester/spark/practica_sdpd2/DATASET-Twitter-23-26-Mar-2014-MotoGP-Qatar.csv",
                        header=True, schema=customSchema, timestampFormat="dd/MM/yyyy HH:mm")
                        #mode="FAILFAST") #

In [16]:
events.printSchema()

root
 |-- Id: long (nullable = true)
 |-- Parent_sys_id: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Mentions: string (nullable = true)
 |-- Target: string (nullable = true)
 |-- Name_source: string (nullable = true)
 |-- Body: string (nullable = true)
 |-- Pub_date: timestamp (nullable = true)
 |-- URLs: string (nullable = true)
 |-- Tipe_action: string (nullable = true)
 |-- Link: string (nullable = true)
 |-- Has_link: byte (nullable = true)
 |-- Has_picture: byte (nullable = true)
 |-- Website: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Activity: long (nullable = true)
 |-- Followers: long (nullable = true)
 |-- Following: long (nullable = true)
 |-- Location: string (nullable = true)



## Ejercicios

### Ejercicio 1

Calcular el número total de tweets por usuario para los 150 usuarios con más mensajes en la muestra de tweets analizados. Calcular, para cada uno de estos usuarios la media de hashtags enviados por mensaje. 
(2.5 puntos)

In [18]:
# CONSULTA 1: Calcular el número total de tweets por usuario para los 150 usuarios con más mensajes.

(events.groupBy("Source")
     .agg(count("Id").alias("tweets"))
     .orderBy("tweets", ascending=False)
     .limit(150).collect())

[Row(Source='m_azharaji', tweets=486),
 Row(Source='twitMOTOGP', tweets=401),
 Row(Source='johnbokke', tweets=297),
 Row(Source='qatarflights', tweets=283),
 Row(Source='box_repsol', tweets=267),
 Row(Source='yolandaa_95', tweets=185),
 Row(Source='AlessiaPont', tweets=182),
 Row(Source='MM93Lovers', tweets=169),
 Row(Source='motomatters', tweets=169),
 Row(Source='Sonic_Moto', tweets=165),
 Row(Source='noelia_260797', tweets=157),
 Row(Source='birtymotogp', tweets=155),
 Row(Source='trackseven707', tweets=152),
 Row(Source='crash_motogp', tweets=149),
 Row(Source='pedrosistas', tweets=148),
 Row(Source='MarcMarquezTeam', tweets=148),
 Row(Source='MotoFamilyGP', tweets=144),
 Row(Source='Kay46_MotoGP', tweets=142),
 Row(Source='blogenboxes', tweets=133),
 Row(Source='Ciintiia93', tweets=133),
 Row(Source='JesiPacio', tweets=130),
 Row(Source='tigrescuba', tweets=125),
 Row(Source='VAVEL_motor', tweets=124),
 Row(Source='MotoGPquotes', tweets=123),
 Row(Source='thalia_26_', tweets=119),

In [19]:
# CONSULTA 2: Calcular, para cada uno de estos usuarios la media de hashtags enviados por mensaje

# Se crea un dataframe con los 150 usuarios con más tweets (se podría coger el de la primera consulta pero, 
# para poder mostrarlo, he decidido volver a crearlo aquí y guardarlo en una variable)

users = (events.groupBy("Source")
             .agg(count("Id").alias("tweets"))
             .orderBy("tweets", ascending=False)
             .limit(150)
             .select("Source").collect())

# Se crea un array vacío donde se guardarán los nombres de los 150 usuarios

Users = []

for r in users:
    Users.append(str(r).replace("Row(Source='", "").replace("')",""))

# Nos quedamos del array original con aquellas filas cuyo usuario se encuentre en el array y se divide el contenido
# de la variable body

df_hashtags = (events.select("Id","Source","Body")
            .filter(events.Source.isin(Users))
            .select("Id","Source",split("Body",' ').alias("Words"))
            .select("Id","Source",explode("Words").alias("Hashtags")))

# Sobre el array, nos quedamos solo con las palabras del body que sean hashtags y procedemos a calcular el número 
# de tweets y el número medio de hashtags por tweet

(df_hashtags.filter(df_hashtags.Hashtags.startswith("#"))
         .groupBy("Source","Id")
         .agg(count("Source").alias("numberHashtags"))
         .groupBy("Source")
         .agg(mean("numberHashtags").alias("meanHashtags"))
         .orderBy("meanHashtags",ascending=False)
         .collect())

[Row(Source='MM93Lovers', meanHashtags=3.808641975308642),
 Row(Source='twitMOTOGP', meanHashtags=3.302325581395349),
 Row(Source='hicarltheprof', meanHashtags=2.872340425531915),
 Row(Source='LiveMoto_GP', meanHashtags=2.5),
 Row(Source='yolandaa_95', meanHashtags=2.4123711340206184),
 Row(Source='EG_00', meanHashtags=2.263157894736842),
 Row(Source='yuliadmy26', meanHashtags=2.210526315789474),
 Row(Source='unmontondruedas', meanHashtags=2.193877551020408),
 Row(Source='ArwikaYulenda', meanHashtags=2.1),
 Row(Source='blogenboxes', meanHashtags=2.0714285714285716),
 Row(Source='Tom_Cruis3', meanHashtags=2.0),
 Row(Source='Martin_Fi3d', meanHashtags=2.0),
 Row(Source='marianav3ga', meanHashtags=2.0),
 Row(Source='MiniD0g', meanHashtags=2.0),
 Row(Source='Silverstonecabs', meanHashtags=2.0),
 Row(Source='Alice_Inferno', meanHashtags=2.0),
 Row(Source='Mari_L0p3z', meanHashtags=2.0),
 Row(Source='DarkSpac3', meanHashtags=2.0),
 Row(Source='MaryPattins0n', meanHashtags=2.0),
 Row(Source='

### Ejercicio 2

Calcular el número total de mensajes que contienen información de geolocalización en el campo LOCATION.
(2.5 puntos)

In [20]:
# Se filtra el contenido de la variable location para quedarnos solo con aquellos cuyo contenido empieza por ÜT, 
# ya que en la ficha técnica pone que solo aquellos que tienen eso, contienen las coordenadas exactas

(events.select("Location").filter(events.Location.startswith('ÜT')).count())        

2136

### Ejercicio 3

Calcular las 5 cuentas de Twitter que más han sido mencionadas en todo el conjunto de datos analizados. 
(2.5 puntos)


In [21]:
# Para poder recoger las menciones será necesario quedarnos únicamente con aquellos tweets en los que se nos
# indique que hay una mención
(events.filter("Tipe_action == 'MT'")
     .select(split("Mentions", ',').alias("allUsers"))
     .select(explode("allUsers").alias("Users"))
     .groupBy("Users")
     .agg(count("Users").alias("countMentions"))
     .orderBy("countMentions", ascending=False)
     .limit(5).show())

+--------------+-------------+
|         Users|countMentions|
+--------------+-------------+
|  valeyellow46|        19599|
| marcmarquez93|        16826|
|        motogp|         7770|
|     lorenzo99|         6969|
|26_danipedrosa|         3093|
+--------------+-------------+



### Ejercicio 4

Calcular los 10 mensajes más retweeteados y los 10 mensajes que han acumulado más respuestas en la muestra de datos analizados. Ahora, restringe la búsqueda a los mensajes en el intervalo 2014-03-24 05:00 - 2014-03-24 11:00. 
(2.5 puntos)

Para esta consulta he dudado acerca de la interpretación, ya que considero que hay dos posibilidades:
La primera es que, sobre los 10 tweets más retweeteados / con más respuestas, se aplica el intervalo para ver cuáles de ellos se produjeron en él. 
La segunda es que se sacan los 10 tweets más retweetedos / con más respuestas que estén dentro del intervalo indicado. 

In [63]:
#PRIMERA INTERPRETACIÓN

# Esta consulta se divide en dos, una para los mensajes retweeteados y otra para los mensajes con más respuestas,
# ambas comparten la misma estructura y en lo único que se diferencian es en el tipo de acción, ya que para la 
# primera consulta nos quedamos con 'RT'y para la segunda 'MT' ('MT' porque cuando se responde un tweet,
# aparece el nombre del usuario al que se responde y, por lo tanto, existe una mención.)
# Nos quedamos con aquellos tweets que tengan padre, es decir, que no son el tweet original, sino que dependen
# de otro, agrupamos por el id del padre que será el tweet original y contamos. 
# Mediante una lista vacía nos quedamos con los id y luego, del dataframe completo, nos quedamos con aquellos
# tweets que se encuentren en la lista y que estén dentro del intervalo deseado. 

# 10 mensajes más retweeteados. En el intervalo temporal indicado solo hay una. 

retweets = (events.select("Parent_sys_id","Tipe_action")
             .filter("Parent_sys_id != 'sin padre'")
             .filter("Tipe_action == 'RT'")
             .groupBy("Parent_sys_id")
             .agg(count("Parent_sys_id").alias("countRetweets"))
             .orderBy("countRetweets",ascending=False)
             .limit(10)
             .select("Parent_sys_id").collect())    
        
Ids = []

for r in retweets:
    Ids.append(str(r).replace("Row(Parent_sys_id='", "").replace("')",""))

(events.select("Id",to_timestamp("Pub_date").alias("Date"),dayofmonth(to_timestamp("Pub_date")).alias("Day"),
                   hour(to_timestamp("Pub_date")).alias("Hour"))
     .filter("Day == 24")
     .filter("(Hour >= 5) AND (Hour < 11)")
     .filter(events.Id.isin(Ids))
     .drop("Day")
     .drop("Hour")
     .show())


# 10 mensajes con más respuestas. En el intervalo temporal indicado no hay ninguna

responses = (events.select("Parent_sys_id","Tipe_action")
             .filter("Parent_sys_id != 'sin padre'")
             .filter("Tipe_action == 'MT'")
             .groupBy("Parent_sys_id")
             .agg(count("Parent_sys_id").alias("countResponses"))
             .orderBy("countResponses",ascending=False)
             .limit(10))

Ids = []

for r in responses:
    Ids.append(str(r).replace("Row(Parent_sys_id='", "").replace("')",""))

(events.select("Id",to_timestamp("Pub_date").alias("Date"),dayofmonth(to_timestamp("Pub_date")).alias("Day"),
                   hour(to_timestamp("Pub_date")).alias("Hour"))
     .filter("Day == 24")
     .filter("(Hour >= 5) AND (Hour < 11)")
     .filter(events.Id.isin(Ids))
     .drop("Day")
     .drop("Hour")
     .show())

+------+-------------------+
|    Id|               Date|
+------+-------------------+
|666385|2014-03-24 10:52:00|
+------+-------------------+

+---+----+
| Id|Date|
+---+----+
+---+----+



In [12]:
# SEGUNDA INTERPRETACIÓN

# Esta consulta se divide en dos, una para los mensajes retweeteados y otra para los mensajes con más respuestas,
# ambas comparten la misma estructura y en lo único que se diferencian es en el tipo de acción, ya que para la 
# primera consulta nos quedamos con 'RT'y para la segunda 'MT' ('MT' porque cuando se responde un tweet,
# aparece el nombre del usuario al que se responde y, por lo tanto, existe una mención.)
# La estructura es similar a la de la primera interpretación pero se incluye el filtrado por hora y día en la 
# primera parte de la consulta, dejando de ser necesaria la segunda parte. 

# 10 mensajes más retweeteados 

(events.select("Parent_sys_id","Tipe_action",dayofmonth(to_timestamp("Pub_date")).alias("Day"),
                   hour(to_timestamp("Pub_date")).alias("Hour"))
         .filter("Parent_sys_id != 'sin padre'")
         .filter("Tipe_action == 'RT'")
         .filter("Day == 24")
         .filter("(Hour >= 5) AND (Hour < 11)")
         .drop("Day")
         .drop("Hour")
         .groupBy("Parent_sys_id")
         .agg(count("Parent_sys_id").alias("countRetweets"))
         .orderBy("countRetweets",ascending=False)
         .limit(10).show())    

# 10 mensajes con más respuestas

(events.select("Parent_sys_id","Tipe_action",dayofmonth(to_timestamp("Pub_date")).alias("Day"),
                   hour(to_timestamp("Pub_date")).alias("Hour"))
         .filter("Parent_sys_id != 'sin padre'")
         .filter("Tipe_action == 'MT'")
         .filter("Day == 24")
         .filter("(Hour >= 5) AND (Hour < 11)")
         .drop("Day")
         .drop("Hour")
         .groupBy("Parent_sys_id")
         .agg(count("Parent_sys_id").alias("countResponses"))
         .orderBy("countResponses",ascending=False)
         .limit(10).show())

+-------------+-------------+
|Parent_sys_id|countRetweets|
+-------------+-------------+
|       603804|          474|
|       645626|          445|
|       638320|          390|
|       666385|          305|
|       645328|          222|
|       605727|          165|
|       622993|          148|
|       635916|          147|
|       553100|          101|
|       645225|           97|
+-------------+-------------+

+-------------+--------------+
|Parent_sys_id|countResponses|
+-------------+--------------+
|       603804|            63|
|       605727|            54|
|       666385|            28|
|       664891|            16|
|       645626|            14|
|       645225|            14|
|       645328|            13|
|       626650|             9|
|       622993|             8|
|       579001|             7|
+-------------+--------------+



## The End

In [22]:
# Remember to stop SparkContext before shutting down this notebook
sc.stop()